In [ ]:
import tensorflow as tf

import nobrainer
from nobrainer import dataset, volume

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

dir_path = os.path.abspath("csv/faced_defaced/train_test_fold_1/csv/")
csv_path = os.path.join(dir_path, "training.csv")
# print(dir_path, csv_path)


labels = pd.read_csv(csv_path)["Y"].values
paths = pd.read_csv(csv_path)["X"].values


# print(labels)

n_classes=2
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)


training_paths = zip(paths, labels)

print(training_paths)

In [2]:
import matplotlib 
# matplotlib.use('Agg')
import os, sys
sys.path.append('../defacing')

from preprocessing.normalization import clip, standardize, normalize
from preprocessing.conform import conform_data
from helpers.utils import load_vol, save_vol, is_gz_file
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import tempfile

In [9]:
import matplotlib 
# matplotlib.use('Agg')
import os, sys
sys.path.append('../defacing')

from preprocessing.normalization import clip, standardize, normalize
from preprocessing.conform import conform_data
from helpers.utils import load_vol, save_vol, is_gz_file
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import tempfile


orig_root_dir = '/work/01329/poldrack/data/mriqc-net/data'
orig_data_face = os.path.join(orig_root_dir, 'face/T1w')
orig_data_mask = os.path.join(orig_root_dir, 'masks')

save_root_dir = '/work/06850/sbansal6/maverick2/mriqc-shared/'

save_preprocessing_face = os.path.join(save_root_dir, 'preprocessing/face')
save_conformed_face = os.path.join(save_root_dir, 'conformed/face')

save_preprocessing_deface = os.path.join(save_root_dir, 'preprocessing/deface')
save_conformed_deface = os.path.join(save_root_dir, 'conformed/deface')

os.makedirs(save_preprocessing_face, exist_ok=True)
os.makedirs(save_preprocessing_deface, exist_ok=True)
os.makedirs(save_conformed_face, exist_ok=True)
os.makedirs(save_conformed_deface, exist_ok=True)

conform_size = (64, 64, 64)
conform_zoom = (4., 4., 4.)


def preprocess(orig_vol_pth, conform_pth, preprocess_pth, DS=None, mask_path=None, debug=False):
    """
    """
    
    filename = orig_vol_pth.split('/')[-1]
    volume, affine, _ = load_vol(orig_vol_pth)
    
    # Preprocessing
    volume = clip(volume, q=90)
    volume = normalize(volume)
    volume = standardize(volume)
    
    # 
    save_preprocessing_path = os.path.join(preprocess_pth, filename)
    save_conformed_path = os.path.join(conform_pth, filename)
    
    print("save_preprocessing_path: ", save_preprocessing_path)
    print("save_conformed_path: ", save_conformed_path)
    
    save_vol(save_preprocessing_path, volume, affine)

    def _plot(data):
        f, axarr = plt.subplots(8, 8, figsize=(12, 12))
        for i in range(8):
            for j in range(8):
                axarr[i, j].imshow(np.rot90(data[:, :, j + 8*i], 1))

        plt.show()
        
#         """
#         plt.subplot(1, 3, 1)
#         plt.imshow(np.rot90(np.mean(data, axis=0)))
#         plt.subplot(1, 3, 2)
#         plt.imshow(np.rot90(np.mean(data, axis=1)))
#         plt.subplot(1, 3, 3)
#         plt.imshow(np.rot90(np.mean(data, axis=2)))
#         plt.show()
#         """
    conform_data(save_preprocessing_path, 
                 out_file=save_conformed_path, 
                 out_size=conform_size, 
                 out_zooms=conform_zoom)

#     if debug: _plot(load_vol(save_conformed_path)[0])
    
    if mask_path and DS:
        mask = np.array(nib.load(mask_path).dataobj)
        masked_volume = volume*mask

        save_mpreprocessing_path = os.path.join(save_preprocessing_deface, DS, filename)
        save_mconformed_path = os.path.join(save_conformed_deface, DS, filename)
        
        print("save_deface_preprocessing_path: ", save_mpreprocessing_path)
        print("save_deface_conformed_path: ", save_mconformed_path)
        
#         os.makedirs(save_mpreprocessing_path, exist_ok=True)
        os.makedirs(os.path.dirname(save_mconformed_path), exist_ok=True)
    
#         save_mpreprocessing_path = os
        
        save_vol(save_mpreprocessing_path, masked_volume, affine)

        conform_data(save_mpreprocessing_path, 
                 out_file=save_mconformed_path, 
                 out_size=conform_size, 
                 out_zooms=conform_zoom)        
        
        return save_conformed_path, save_mconformed_path

    return save_conformed_path


def checkNonConformed(orig_path, save_path):

    conform = []
    orig = []

    for path in glob(save_path + "/*/*.nii*"):
        tempname = path.split("/")[-1]
        ds = path.split("/")[-2]
        conform.append(ds + "/" + tempname)

    print("Total Conformed: ", len(conform))

    for path in glob(orig_path + "/*/*.nii*"):
        tempname = path.split("/")[-1]
        ds = path.split("/")[-2]
        orig.append(ds + "/" + tempname)

    print("Total Original: ", len(orig))

    print("Total not conformed: ", len(orig) - len(conform))

    count = 0
    for f in orig:
        exists = False
        for fc in conform:
            if fc in f:
                exists = True
        if not exists:
            count += 1
            print("Not conformed file: ", f)


for path in glob(orig_data_face + "/*/*.nii*"):
    try:
        if 'ds000140_anat' not in path:
            print("Orig Path: ", path)
            # Example: 
            #        vol_name - sub-22_T1w.nii.gz
            #        DS - ds000140_anat
            vol_name = path.split("/")[-1]
            DS = path.split("/")[-2]
            
            
                
            # directories for saving preprocessed and conformed volumes
            ds_save_conform_path = os.path.join(save_conformed_face, DS)
            ds_save_preprocess_path = os.path.join(save_preprocessing_face, DS)

            # Get the mask path
            mask_path = glob(os.path.join(orig_data_mask, DS, vol_name.split('.')[0] + "*_mask*"))[0]

            print("Mask_path", mask_path)

            if not os.path.exists(ds_save_conform_path):
                os.makedirs(ds_save_conform_path)

            if not os.path.exists(ds_save_preprocess_path):
                os.makedirs(ds_save_preprocess_path)

            # Check if volume is a proper gunzipped
            if not os.path.splitext(path)[1] == ".gz" and is_gz_file(path):
                rename_file = os.path.splitext(vol_name)[0]
                fixed_gz_tmp = os.path.join(save_conformed_face, rename_file)
                print(fixed_gz_tmp)
                subprocess.call(["cp", path, fixed_gz_tmp])

                print(preprocess(fixed_gz_tmp,
                                 ds_save_conform_path,
                                 ds_save_preprocess_path,
                                 mask_path = mask_path,
                                 DS=DS
                                ))
                os.remove(fixed_gz_tmp)

            else:
                print(preprocess(path,
                                 ds_save_conform_path,
                                 ds_save_preprocess_path,
                                 mask_path = mask_path,
                                 DS=DS
                                ))
    except:
        print("Preprocessing incomplete. Exception occurred.")
        pass


# for path in glob(orig_data_deface + "/*/*.nii*"):
#     try:
#         print("Orig Path: ", path)
#         if not is_gz_file(path) and os.path.splitext(path)[1] == ".gz":
#             tempname = path.split("/")[-1]
#             ds = path.split("/")[-2]
#             rename_file = os.path.splitext(tempname)[0]
#             dst = os.path.join(save_data_deface, rename_file)
# #             print(dst)
#             subprocess.call(["cp", path, dst])
#             ds_save_path = os.path.join(save_data_deface, ds)
#             if not os.path.exists(ds_save_path):
#                 os.makedirs(ds_save_path)
#             preprocess(dst, conform_size, save_data_path=ds_save_path))
#         else:
#             ds = path.split("/")[-2]
#             ds_save_path = os.path.join(save_data_deface, ds)
#             if not os.path.exists(ds_save_path):
#                 os.makedirs(ds_save_path)
#             preprocess(path, conform_size, save_data_path=ds_save_path)
#     except:
#         print("Preprocessing incomplete. Exception occurred.")
#         pass
    

# checkNonConformed(orig_data_face, save_data_face)
# checkNonConformed(orig_data_deface, save_data_deface)


Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000119_anat/sub-10_T1w.nii.gz
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000119_anat/sub-10_T1w_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-10_T1w.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-10_T1w.nii.gz
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000119_anat/sub-10_T1w.nii.gz
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-10_T1w.nii.gz
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-10_T1w.nii.gz', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-10_T1w.nii.gz')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000119_anat/sub-07_T1w.nii.gz
Mask_path /work/01329/poldrack/d

save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000119_anat/sub-22_T1w.nii.gz
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-22_T1w.nii.gz
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-22_T1w.nii.gz', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-22_T1w.nii.gz')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000119_anat/sub-52_T1w.nii.gz
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000119_anat/sub-52_T1w_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-52_T1w.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-52_T1w.nii.gz
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000119_anat/sub-52_T1w.nii

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-69_T1w.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-69_T1w.nii.gz
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000119_anat/sub-69_T1w.nii.gz
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-69_T1w.nii.gz
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-69_T1w.nii.gz', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-69_T1w.nii.gz')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000119_anat/sub-26_T1w.nii.gz
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000119_anat/sub-26_T1w_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-26_T1w.nii.gz
save_

('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-24_T1w.nii.gz', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-24_T1w.nii.gz')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000119_anat/sub-17_T1w.nii.gz
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000119_anat/sub-17_T1w_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-17_T1w.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-17_T1w.nii.gz
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000119_anat/sub-17_T1w.nii.gz
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-17_T1w.nii.gz
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-17_T1w.nii.gz', '/work/06850/sbansal6/maverick2

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-63_T1w.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-63_T1w.nii.gz
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000119_anat/sub-63_T1w.nii.gz
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-63_T1w.nii.gz
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000119_anat/sub-63_T1w.nii.gz', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000119_anat/sub-63_T1w.nii.gz')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000119_anat/sub-21_T1w.nii.gz
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000119_anat/sub-21_T1w_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000119_anat/sub-21_T1w.nii.gz
save_

...

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000206_anat/sub-THP0002_ses-THP0002IOWA2_acq-T130_run-002_T1w.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000206_anat/sub-THP0002_ses-THP0002IOWA2_acq-T130_run-002_T1w.nii.gz
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000206_anat/sub-THP0002_ses-THP0002IOWA2_acq-T130_run-002_T1w.nii.gz
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000206_anat/sub-THP0002_ses-THP0002IOWA2_acq-T130_run-002_T1w.nii.gz
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000206_anat/sub-THP0002_ses-THP0002IOWA2_acq-T130_run-002_T1w.nii.gz', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000206_anat/sub-THP0002_ses-THP0002IOWA2_acq-T130_run-002_T1w.nii.gz')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000206_anat/sub-THP0003_s

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000213_anat/sub-09_T1map.nii
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-09_T1map.nii
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000213_anat/sub-09_T1map.nii
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000213_anat/sub-09_T1map.nii
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-09_T1map.nii', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000213_anat/sub-09_T1map.nii')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000213_anat/sub-12_T1map.nii
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-12_T1map_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000213_anat/sub-12_T1map.nii
save_confor

('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-22_T1map.nii', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000213_anat/sub-22_T1map.nii')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000213_anat/sub-25_T1map.nii
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-25_T1map_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000213_anat/sub-25_T1map.nii
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-25_T1map.nii
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000213_anat/sub-25_T1map.nii
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000213_anat/sub-25_T1map.nii
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-25_T1map.nii', '/work/06850/sbansal6/maverick2/mriqc

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000213_anat/sub-19_T1map.nii
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-19_T1map.nii
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000213_anat/sub-19_T1map.nii
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000213_anat/sub-19_T1map.nii
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000213_anat/sub-19_T1map.nii', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000213_anat/sub-19_T1map.nii')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000213_anat/sub-01_T1map.nii
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-01_T1map_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000213_anat/sub-01_T1map.nii
save_confor

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000159_anat/sub-44_T1w.nii
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000159_anat/sub-44_T1w.nii
save_deface_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/deface/ds000159_anat/sub-44_T1w.nii
save_deface_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000159_anat/sub-44_T1w.nii
('/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face/ds000159_anat/sub-44_T1w.nii', '/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface/ds000159_anat/sub-44_T1w.nii')
Orig Path:  /work/01329/poldrack/data/mriqc-net/data/face/T1w/ds000159_anat/sub-43_T1w.nii
Mask_path /work/01329/poldrack/data/mriqc-net/data/masks/ds000159_anat/sub-43_T1w_pydeface_mask.nii.gz
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face/ds000159_anat/sub-43_T1w.nii
save_conformed_path:  /work/0

In [13]:
checkNonConformed(orig_data_deface, save_conformed_deface)

NameError: name 'orig_data_deface' is not defined

In [ ]:
face_C = []
face_O = []

for path in glob(save_data_face + "/*/*.nii*"):
    tempname = path.split("/")[-1]
    ds = path.split("/")[-2]
    face_C.append(ds + '/' + tempname)

print(len(face_C))
# print(face_C)


for path in glob(orig_data_face + "/*/*.nii*"):
    tempname = path.split("/")[-1]
    ds = path.split("/")[-2]
    face_O.append(ds + '/' + tempname)

print(len(face_O))
# print(face_O)

count = 0
for f in face_O:
    exists = False
    for fc in face_C:
        if fc in f:
            exists = True
    if not exists:
        count += 1
        print(f)
print(count)

In [ ]:
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd

face_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face"
deface_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface"

paths = []
labels = []

for path in glob(deface_path + "/*/*.nii*"):
    paths.append(path)
    labels.append(0)

for path in glob(face_path + "/*/*.nii*"):
    paths.append(path)
    labels.append(1)
    
print(len(paths))
print(len(labels))

save_path = "/work/06850/sbansal6/maverick2/mriqc-shared/csv"

os.makedirs(save_path, exist_ok=True)

df = pd.DataFrame()
df["X"] = paths
df["Y"] = labels
df.to_csv(os.path.join(save_path, "all.csv"))

SPLITS = 10
skf = StratifiedKFold(n_splits=SPLITS)
fold_no = 1

for train_index, test_index in skf.split(paths, labels):
    out_path = save_path + "/train_test_fold_{}/csv/".format(fold_no)

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    image_train, image_test = (
        itemgetter(*train_index)(paths),
        itemgetter(*test_index)(paths),
    )
    label_train, label_test = (
        itemgetter(*train_index)(labels),
        itemgetter(*test_index)(labels),
    )

    # image_train = [os.path.join(data_path, 'sub-' + str(pth) + '_T1w.nii.gz') for pth in image_train]
    train_data = {"X": image_train, "Y": label_train}
    df_train = pd.DataFrame(train_data)
    df_train.to_csv(os.path.join(out_path, "training.csv"), index=False)

    # image_test = [os.path.join(data_path, 'sub-' + str(pth) + '_T1w.nii.gz') for pth in image_test]
    validation_data = {"X": image_test, "Y": label_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(out_path, "validation.csv"), index=False)

    fold_no += 1

In [ ]:
import random
import nobrainer
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
import pandas as pd

SPLITS = 10

for fold in range(1, SPLITS+1):
    
    dir_path = "/work/06850/sbansal6/maverick2/mriqc-shared/csv/train_test_fold_{}/csv/".format(fold)
    
    tf_records_dir = "/work/06850/sbansal6/maverick2/mriqc-shared/tfrecords/tfrecords_fold_{}/".format(fold)
    os.makedirs(tf_records_dir, exist_ok=True)
    
    train_csv_path = os.path.join(dir_path, "training.csv")
    valid_csv_path = os.path.join(dir_path, "validation.csv")
    
    train_paths = pd.read_csv(train_csv_path)["X"].values
    train_labels = pd.read_csv(train_csv_path)["Y"].values
    train_D = list(zip(train_paths, train_labels))
    random.shuffle(train_D)
#     print(train_D[0])
    
    valid_paths = pd.read_csv(valid_csv_path)["X"].values
    valid_labels = pd.read_csv(valid_csv_path)["Y"].values
    valid_D = list(zip(valid_paths, valid_labels))
    random.shuffle(valid_D)
    
    train_write_path = os.path.join(tf_records_dir, 'data-train_shard-{shard:03d}.tfrec')
    valid_write_path = os.path.join(tf_records_dir, 'data-valid_shard-{shard:03d}.tfrec')
    
    nobrainer.tfrecord.write(
        features_labels=train_D,
        filename_template=train_write_path,
        examples_per_shard=3)
    
    nobrainer.tfrecord.write(
        features_labels=valid_D,
        filename_template=valid_write_path,
        examples_per_shard=1)

In [ ]:
print(tf.__version__)

In [2]:
import nobrainer
from nobrainer.io import _is_gzipped
from nobrainer.volume import to_blocks
import sys

sys.path.append("..")
from helpers.utils import load_vol
import tensorflow_probability as tfp
import tensorflow as tf
import glob
import numpy as np

AUTOTUNE = tf.data.experimental.AUTOTUNE
ROOTDIR = "/work/06850/sbansal6/maverick2/mriqc-shared/"

DISTRIBUTION = load_vol("../defacing/helpers/distribution.nii.gz")[0]
DISTRIBUTION /= DISTRIBUTION.sum()

sampler = lambda n: np.array(
    [
        np.unravel_index(
            np.random.choice(
                np.arange(np.prod(DISTRIBUTION.shape)), p=DISTRIBUTION.ravel()
            ),
            DISTRIBUTION.shape,
        )
        for _ in range(n)
    ]
)


# function to apply augmentations to tf dataset
def apply_augmentations(features, labels):

    """ Apply <TYPE_OF> augmentation to the dataset

    """
    #     iaa.SomeOf(
    #             (0, 3),
    #             [
    #                 iaa.Fliplr(0.5),
    #                 iaa.Flipud(0.5),
    #                 iaa.Noop(),
    #                 iaa.OneOf(
    #                     [
    #                         iaa.Affine(rotate=90),
    #                         iaa.Affine(rotate=180),
    #                         iaa.Affine(rotate=270),
    #                     ]
    #                 ),
    #                 # iaa.GaussianBlur(sigma=(0.0, 0.2)),
    #             ],
    #         )

    return


def _magic_slicing_(shape):
    """
    """
    idx = []
    for ii in np.arange(shape[0]):
        if (ii % shape[0] ** 0.5) == 0:
            idx.append(ii)
    idx = np.array(idx)
    return idx


def get_dataset(
    file_pattern,
    n_classes,
    batch_size,
    volume_shape,
    plane,
    n=24,
    block_shape=None,
    n_epochs=None,
    mapping=None,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=AUTOTUNE,
):

    """ Returns tf.data.Dataset after preprocessing from
    tfrecords for training and validation

    Parameters
    ----------
    file_pattern:

    n_classes:

    """

    files = glob.glob(file_pattern)

    if not files:
        raise ValueError("no files found for pattern '{}'".format(file_pattern))

    compressed = _is_gzipped(files[0])
    shuffle = bool(shuffle_buffer_size)

    ds = nobrainer.dataset.tfrecord_dataset(
        file_pattern=file_pattern,
        volume_shape=volume_shape,
        shuffle=shuffle,
        scalar_label=True,
        compressed=compressed,
        num_parallel_calls=num_parallel_calls,
    )

    if augment:
        ds = ds.map(
            lambda x, y: tf.cond(
                tf.random.uniform((1,)) > 0.5,
                true_fn=lambda: apply_augmentations(x, y),
                false_fn=lambda: (x, y),
            ),
            num_parallel_calls=num_parallel_calls,
        )

    def _ss(x, y):
        x, y = structural_slice(x, y, plane, n)
        return (x, y)

    ds = ds.map(_ss, num_parallel_calls)

    #     def _f(x, y):
    #         x = to_blocks(x, block_shape)
    #         n_blocks = x.shape[0]
    #         y = tf.repeat(y, n_blocks)
    #         return (x, y)
    #     ds = ds.map(_f, num_parallel_calls=num_parallel_calls)

    # This step is necessary because it reduces the extra dimension.
    # ds = ds.unbatch()

    ds = ds.prefetch(buffer_size=batch_size)

    def reshape(x, y):
        if plane == "combined":
            for _ in 3:
                pass
        return (x, y)

    if batch_size is not None:
        ds = ds.batch(batch_size=batch_size, drop_remainder=True)

    if shuffle_buffer_size:
        ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat the dataset n_epochs times
    ds = ds.repeat(n_epochs)

    return ds


def structural_slice(x, y, plane, n=4):

    """ Transpose dataset based on the plane

    Parameters
    ----------
    x:

    y:

    plane:

    """

    options = ["axial", "coronal", "sagittal", "combined"]
    shape = np.array(x.shape)

    if isinstance(plane, str) and plane in options:
        if plane == "axial":
            idx = np.random.randint(shape[0] ** 0.5)
            midx = sampler(n)[:, 0]
            x = x
            k = 3

        if plane == "coronal":
            idx = np.random.randint(shape[1] ** 0.5)
            midx = sampler(n)[:, 1]
            x = tf.transpose(x, perm=[1, 2, 0])
            k = 2

        if plane == "sagittal":
            idx = np.random.randint(shape[2] ** 0.5)
            midx = sampler(n)[:, 2]
            x = tf.transpose(x, perm=[2, 0, 1])
            k = 1

        if plane == "combined":
            temp = {}
            for op in options[:-1]:
                temp[op] = structural_slice(x, y, op, n)[0]
            x = temp

        if not plane == "combined":
            x = tf.squeeze(tf.gather_nd(x, midx.reshape(n, 1, 1)), axis=1)
            x = tf.math.reduce_mean(x, axis=0)
            x = tf.convert_to_tensor(tf.expand_dims(x, axis=-1))
            x = tf.image.rot90(x, k, name=None)

        # y = tf.repeat(y, n)

        return x, y
    else:
        raise ValueError("expected plane to be one of ['axial', 'coronal', 'sagittal']")


# if __name__ == "__main__":
    
#     n_classes = 2
#     global_batch_size = 8
#     volume_shape = (64, 64, 64)
#     ds = get_dataset(
#         ROOTDIR + "tfrecords_F15/tfrecords_fold_1/data-train_*",
#         n_classes=n_classes,
#         batch_size=global_batch_size,
#         volume_shape=volume_shape,
#         plane="combined",
#         shuffle_buffer_size=3,
#     )

#     import matplotlib

#     matplotlib.use("Agg")
#     import matplotlib.pyplot as plt

#     print(ds)
#     for ii in range(100):
#         x, y = next(ds.as_numpy_iterator())
#         # print (np.min(x), np.max(x), np.unique(y))
#         count = 1
#         for i in range(global_batch_size):
#             for key in x.keys():
#                 plt.subplot(global_batch_size, 3, count)
#                 plt.imshow(x[key][i, :, :, 0])
#                 plt.title(str(y[i]))
#                 plt.xticks([], " ")
#                 plt.yticks([], " ")
#                 count += 1
#         plt.savefig("processed_image_combined_{}.png".format(ii))


# dataset_train_coronal = get_dataset("tfrecords/tfrecords_fold_1/data-train_*",
#                             n_classes=n_classes,
#                             batch_size=global_batch_size,
#                             volume_shape=volume_shape,
#                             block_shape=block_shape,
#                             plane='coronal',
#                             shuffle_buffer_size=3)

# dataset_train_sagittal = get_dataset("tfrecords/tfrecords_fold_1/data-train_*",
#                             n_classes=n_classes,
#                             batch_size=global_batch_size,
#                             volume_shape=volume_shape,
#                             block_shape=block_shape,
#                             plane='sagittal',
#                             shuffle_buffer_size=3)


In [4]:
# Std packages
import sys, os
import glob
import math

sys.path.append("../defacing")

# Custom packages
from models import modelN
# from dataloaders.dataset import get_dataset

# Tf packages
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
)
# import nobrainer
from tensorflow.keras import metrics
from tensorflow.keras import losses

ROOTDIR = "/work/06850/sbansal6/maverick2/mriqc-shared/"


def scheduler(epoch):
    if epoch < 3:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))


def train(
    volume_shape=(64, 64, 64),
    image_size=(64, 64),
    dropout=0.2,
    batch_size=8,
    n_classes=2,
    n_epochs=30,
    fold=1
):
    tpaths = glob.glob(os.path.join(ROOTDIR, "tfrecords_F15/tfrecords_fold_{}/data-train_*".format(fold)))
    vpaths = glob.glob(os.path.join(ROOTDIR, "tfrecords_F15/tfrecords_fold_{}/data-valid_*".format(fold)))

    planes = ["axial", "coronal", "sagittal", "combined"]

    strategy = tf.distribute.MirroredStrategy()
    BATCH_SIZE_PER_REPLICA = batch_size
    global_batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

    model_save_path = os.path.join(ROOTDIR, "model_save_dir_F15/folds_{}".format(fold))
    
    os.makedirs(model_save_path, exist_ok=True)

    cp_save_path = os.path.join(model_save_path, "weights")

    logdir_path = os.path.join(model_save_path, "tb_logs")
    if not os.path.exists(logdir_path):
        os.makedirs(logdir_path)

    for plane in planes:

        logdir = os.path.join(logdir_path, plane)
        os.makedirs(logdir, exist_ok=True)

        tbCallback = TensorBoard(
            log_dir=logdir, histogram_freq=0, write_graph=True, write_images=False,
        )

        os.makedirs(os.path.join(cp_save_path, plane), exist_ok=True)

        model_checkpoint = ModelCheckpoint(
            os.path.join(cp_save_path, plane, "best-wts.h5"),
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            mode="min",
        )

        with strategy.scope():

            if not plane == "combined": 
                lr = 1e-3
                model = modelN.Submodel(
                    input_shape=image_size,
                    dropout=dropout,
                    name=plane,
                    include_top=True,
                    weights=None,
                )
            else:
                lr = 5e-4
                model = modelN.CombinedClassifier(
                    input_shape=image_size,
                    dropout=dropout,
                    trainable=True,
                    wts_root=cp_save_path,
                )

            print("Submodel: ", plane)
            print(model.summary())

            METRICS = [
                metrics.TruePositives(name="tp"),
                metrics.FalsePositives(name="fp"),
                metrics.TrueNegatives(name="tn"),
                metrics.FalseNegatives(name="fn"),
                metrics.BinaryAccuracy(name="accuracy"),
                metrics.Precision(name="precision"),
                metrics.Recall(name="recall"),
                metrics.AUC(name="auc"),
            ]

            model.compile(
                loss=tf.keras.losses.binary_crossentropy,
                optimizer=Adam(learning_rate=lr),
                metrics=METRICS,
            )

        print("GLOBAL BATCH SIZE: ", global_batch_size)

        dataset_train = get_dataset(
            file_pattern=os.path.join(ROOTDIR, "tfrecords_F15/tfrecords_fold_{}/data-train_*".format(fold)),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            shuffle_buffer_size=global_batch_size,
        )

        dataset_valid = get_dataset(
            file_pattern=os.path.join(ROOTDIR, "tfrecords_F15/tfrecords_fold_{}/data-valid_*".format(fold)),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            shuffle_buffer_size=global_batch_size,
        )

        steps_per_epoch = math.ceil(len(tpaths)/batch_size)

        validation_steps = math.ceil(len(vpaths)/batch_size)

        print(steps_per_epoch, validation_steps)

        lrcallback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        
        model.fit(
            dataset_train,
            epochs=n_epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=dataset_valid,
            validation_steps=validation_steps,
            callbacks=[tbCallback, model_checkpoint],
        )

        del model
        K.clear_session()


if __name__ == "__main__":
    for fold in range(1, 16):
        train(fold=fold)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Submodel:  axial
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
axial (InputLayer)           [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 64, 8)         80        
_________________________________________________________________
batch_normalization_12 (Batc (None, 64, 64, 8)         32        
_________________________________________________________________
activation_12 (Activation)   (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 64, 8)         584   

Epoch 10/30
79/79 [==============================] - 4s 47ms/step - loss: 0.0115 - tp: 1238.0000 - fp: 6.0000 - tn: 1276.0000 - fn: 8.0000 - accuracy: 0.9945 - precision: 0.9952 - recall: 0.9936 - auc: 0.9999 - val_loss: 5.6417e-04 - val_tp: 271.0000 - val_fp: 0.0000e+00 - val_tn: 273.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 11/30
79/79 [==============================] - 4s 47ms/step - loss: 0.0072 - tp: 1288.0000 - fp: 3.0000 - tn: 1231.0000 - fn: 6.0000 - accuracy: 0.9964 - precision: 0.9977 - recall: 0.9954 - auc: 1.0000 - val_loss: 5.4729e-05 - val_tp: 272.0000 - val_fp: 0.0000e+00 - val_tn: 272.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 12/30
79/79 [==============================] - 4s 46ms/step - loss: 0.0090 - tp: 1243.0000 - fp: 3.0000 - tn: 1279.0000 - fn: 3.0000 - accuracy: 0.9976 - precision: 0.9976 - recall: 0.9976 - auc: 0.

Submodel:  coronal
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
coronal (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 8)         80        
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 8)         32        
_________________________________________________________________
activation (Activation)      (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 8)         584       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64

79/79 [==============================] - 4s 51ms/step - loss: 0.0135 - tp: 1282.0000 - fp: 4.0000 - tn: 1235.0000 - fn: 7.0000 - accuracy: 0.9956 - precision: 0.9969 - recall: 0.9946 - auc: 0.9999 - val_loss: 0.0062 - val_tp: 265.0000 - val_fp: 0.0000e+00 - val_tn: 275.0000 - val_fn: 4.0000 - val_accuracy: 0.9926 - val_precision: 1.0000 - val_recall: 0.9851 - val_auc: 1.0000
Epoch 12/30
79/79 [==============================] - 4s 51ms/step - loss: 0.0145 - tp: 1246.0000 - fp: 7.0000 - tn: 1268.0000 - fn: 7.0000 - accuracy: 0.9945 - precision: 0.9944 - recall: 0.9944 - auc: 0.9999 - val_loss: 0.0124 - val_tp: 266.0000 - val_fp: 0.0000e+00 - val_tn: 274.0000 - val_fn: 4.0000 - val_accuracy: 0.9926 - val_precision: 1.0000 - val_recall: 0.9852 - val_auc: 1.0000
Epoch 13/30
79/79 [==============================] - 4s 52ms/step - loss: 0.0235 - tp: 1264.0000 - fp: 14.0000 - tn: 1242.0000 - fn: 8.0000 - accuracy: 0.9913 - precision: 0.9890 - recall: 0.9937 - auc: 0.9996 - val_loss: 0.0252 - v

GLOBAL BATCH SIZE:  32
79 17
Train for 79 steps, validate for 17 steps
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 28 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 28 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
79/79 [==============================] - 19s 247ms/step - loss: 0.2647 - tp: 1152.0000 - fp: 96.0000 - tn: 1157.0000 - fn: 123.0000 - accuracy: 0.9134 - precision: 0.9231 - recall: 0.9035 - auc: 0.9669 - val_loss: 0.6497 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 274.0000 - val_fn: 270.0000 - val_accuracy: 0.5037 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 1.0000
Epoch 2/30
79/79 [==============================] - 5s 60ms/step - loss: 0.0540 - tp: 1236.0000 - fp: 18.0000 - tn: 1255.0000 - fn: 19.0000 - accuracy: 0.9854 - precision: 0.9856 - recall: 0.9849 - auc: 0.9973 - val_los

...

Epoch 5/30
80/80 [==============================] - 4s 53ms/step - loss: 0.0242 - tp: 1279.0000 - fp: 13.0000 - tn: 1258.0000 - fn: 10.0000 - accuracy: 0.9910 - precision: 0.9899 - recall: 0.9922 - auc: 0.9997 - val_loss: 0.0081 - val_tp: 273.0000 - val_fp: 0.0000e+00 - val_tn: 271.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 6/30
80/80 [==============================] - 4s 52ms/step - loss: 0.0324 - tp: 1270.0000 - fp: 17.0000 - tn: 1254.0000 - fn: 19.0000 - accuracy: 0.9859 - precision: 0.9868 - recall: 0.9853 - auc: 0.9994 - val_loss: 0.0029 - val_tp: 274.0000 - val_fp: 0.0000e+00 - val_tn: 270.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 7/30
80/80 [==============================] - 4s 52ms/step - loss: 0.0253 - tp: 1265.0000 - fp: 10.0000 - tn: 1273.0000 - fn: 12.0000 - accuracy: 0.9914 - precision: 0.9922 - recall: 0.9906 - auc: 0.9996 

Epoch 26/30
80/80 [==============================] - 4s 52ms/step - loss: 0.0082 - tp: 1260.0000 - fp: 6.0000 - tn: 1289.0000 - fn: 5.0000 - accuracy: 0.9957 - precision: 0.9953 - recall: 0.9960 - auc: 1.0000 - val_loss: 1.4832e-04 - val_tp: 270.0000 - val_fp: 0.0000e+00 - val_tn: 274.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 27/30
80/80 [==============================] - 4s 53ms/step - loss: 0.0078 - tp: 1273.0000 - fp: 7.0000 - tn: 1274.0000 - fn: 6.0000 - accuracy: 0.9949 - precision: 0.9945 - recall: 0.9953 - auc: 1.0000 - val_loss: 9.5641e-05 - val_tp: 273.0000 - val_fp: 0.0000e+00 - val_tn: 271.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 28/30
80/80 [==============================] - 4s 54ms/step - loss: 0.0063 - tp: 1289.0000 - fp: 5.0000 - tn: 1265.0000 - fn: 1.0000 - accuracy: 0.9977 - precision: 0.9961 - recall: 0.9992 - auc: 1.

GLOBAL BATCH SIZE:  32
80 17
Train for 80 steps, validate for 17 steps
Epoch 1/30
80/80 [==============================] - 29s 363ms/step - loss: 0.4640 - tp: 1146.0000 - fp: 146.0000 - tn: 1133.0000 - fn: 135.0000 - accuracy: 0.8902 - precision: 0.8870 - recall: 0.8946 - auc: 0.9352 - val_loss: 0.6077 - val_tp: 20.0000 - val_fp: 0.0000e+00 - val_tn: 264.0000 - val_fn: 260.0000 - val_accuracy: 0.5221 - val_precision: 1.0000 - val_recall: 0.0714 - val_auc: 1.0000
Epoch 2/30
80/80 [==============================] - 5s 63ms/step - loss: 0.0495 - tp: 1262.0000 - fp: 19.0000 - tn: 1261.0000 - fn: 18.0000 - accuracy: 0.9855 - precision: 0.9852 - recall: 0.9859 - auc: 0.9978 - val_loss: 0.5466 - val_tp: 30.0000 - val_fp: 0.0000e+00 - val_tn: 273.0000 - val_fn: 241.0000 - val_accuracy: 0.5570 - val_precision: 1.0000 - val_recall: 0.1107 - val_auc: 0.9985
Epoch 3/30
80/80 [==============================] - 5s 66ms/step - loss: 0.0388 - tp: 1255.0000 - fp: 13.0000 - tn: 1274.0000 - fn: 18.0000 -

Epoch 21/30
80/80 [==============================] - 5s 64ms/step - loss: 0.0149 - tp: 1276.0000 - fp: 9.0000 - tn: 1269.0000 - fn: 6.0000 - accuracy: 0.9941 - precision: 0.9930 - recall: 0.9953 - auc: 0.9995 - val_loss: 0.0029 - val_tp: 270.0000 - val_fp: 0.0000e+00 - val_tn: 274.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 22/30
80/80 [==============================] - 5s 65ms/step - loss: 0.0099 - tp: 1267.0000 - fp: 6.0000 - tn: 1281.0000 - fn: 6.0000 - accuracy: 0.9953 - precision: 0.9953 - recall: 0.9953 - auc: 0.9999 - val_loss: 0.0024 - val_tp: 270.0000 - val_fp: 0.0000e+00 - val_tn: 274.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 23/30
80/80 [==============================] - 5s 61ms/step - loss: 0.0079 - tp: 1269.0000 - fp: 4.0000 - tn: 1283.0000 - fn: 4.0000 - accuracy: 0.9969 - precision: 0.9969 - recall: 0.9969 - auc: 1.0000 - v

KeyboardInterrupt: 

In [ ]:
!pip install tensorflow-gpu==2.1

In [ ]:
tf.__version__